In [23]:
import pandas as pd
import glob
import lxml.html
import random

In [24]:
random.random()

0.37960909731866566

In [25]:
a = ['a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c']
g = [1 if random.random() < 0.5 else 0 for i in a]

In [26]:
g

[0, 1, 1, 0, 1, 0, 0, 1, 1]

In [27]:
def concatenate_gpt(files):
    dfs = []
    for file in files:
        raw_df = pd.read_csv(file)
        df = pd.DataFrame({'text': [str(t).strip() for t in raw_df['text']],
                       'generated': 1})
        dfs.append(df)

    gpt = pd.concat(dfs, ignore_index=True)
    gpt.to_csv('gpt.csv', index=False)
    return gpt


In [28]:
concatenate_gpt(glob.glob('gpt_data/*.csv'))

,text,generated
0,"Dawn on its way to victory, but where does it ...",1
1,Drew Angerer/Getty Images\n\nThere are many th...,1
2,"In my last post, I gave a post on the use of t...",1
3,Brought to you by:\n\nKFC\n\nHogan's Day\n\nSo...,1
4,"It's time for the first time since 2008, and w...",1
...,...,...
9995,"alabamer, philly pix, pestle, live nectar, dan...",1
9996,Ballistics System Summary\n\nA linear three-di...,1
9997,http://dailytarbadilla.com/policies/opinions/2...,1
9998,"Harmony in that regard is momentous'"", Ocaden ...",1


In [29]:
def parse_blog_xml(file):
    tree = lxml.html.parse(file)
    post_els = tree.findall('.//post')
    posts = [p.text.replace('urlLink', '').strip() for p in post_els]
    return posts

In [30]:
BLOG_FILE_NUM = 0 # change this to add blog data
def process_human(fake=False):
    dfs = []

    files = glob.glob('nyt_data/*.csv')

    for file in files:
        raw_df = pd.read_csv(file)
        text_list = [str(t).strip() for t in raw_df['abstract']]
        generated = [1 if random.random() < 0.5 else 0 for _ in text_list] if fake else 0
        df = pd.DataFrame({'text': text_list,
                       'generated': generated})
        dfs.append(df)

    nyt_scraped_files = glob.glob('../nyt_scraper/data/*.txt')
    for file in nyt_scraped_files:
        lines = open(file).readlines()
        text_list = [l.strip() for l in lines]
        generated = [1 if random.random() < 0.5 else 0 for _ in text_list] if fake else 0
        df = pd.DataFrame({'text': [l.strip() for l in lines],
                       'generated': generated})
        dfs.append(df)

    blog_files = glob.glob('blogs/*.xml')[:BLOG_FILE_NUM]
    for file in blog_files:
        posts = parse_blog_xml(file)
        generated = [1 if random.random() < 0.5 else 0 for _ in posts] if fake else 0
        df = pd.DataFrame({'text': posts,
                       'generated': generated})
        dfs.append(df)

    human = pd.concat(dfs, ignore_index=True)
    outfile = 'fake.csv' if fake else 'human.csv'
    human.to_csv(outfile, index=False)
    return human

In [31]:
process_human(True)

,text,generated
0,Congress could do much more to protect America...,1.0
1,Christina Iverson and Jeff Chen ring in the Ne...,0.0
2,"All year long, Earth passes through streams of...",1.0
3,"Never miss an eclipse, a meteor shower, a rock...",1.0
4,A year full of highs and lows in space just en...,0.0
...,...,...
119985,"The program, The Globe wrote, was “a penetrati...",0.0
119986,Ms. Jarvis began filming in China in August 19...,0.0
119987,Ms. Jarvis left NBC in 1976 and founded her ow...,0.0
119988,"In addition to “Junon and Avos,” which Ms. Jar...",0.0
